In [83]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
import hvplot.pandas
%matplotlib inline

In [84]:
pd.set_option('display.max_rows', 3200)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [85]:
etf_portfolio = pd.DataFrame(index=["purchase_date", "weights", "seed_money", "num_shares"],
                            columns = ['XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLRE', 'XLU', 'XLV',
       'XLY'])
etf_portfolio

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY
purchase_date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
weights,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
seed_money,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
num_shares,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
df = pd.read_csv("etf_daily_prices.csv", index_col='Date', parse_dates=True, infer_datetime_format=True)

In [87]:
df = df.loc['2018-06-29':]
df.head()

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY
Date,,,,,,,,,,,
2018-06-29,58.07,49.4700,75.98,26.57,71.64,69.43,51.51,32.71,51.94,83.44,109.29
2018-07-02,57.77,49.8400,74.75,26.72,71.76,70.10,51.23,32.53,52.33,83.70,109.45
2018-07-03,57.67,49.1890,75.37,26.47,71.56,69.27,51.36,32.71,52.48,83.96,108.89
2018-07-05,58.17,49.9799,75.21,26.54,71.89,70.24,52.08,33.14,52.79,84.82,109.45
2018-07-06,58.44,50.7700,75.68,26.66,72.15,71.05,52.23,33.27,53.18,86.02,110.31


In [88]:
weight = pd.read_csv("berkshire_weighting_data.csv", index_col = 'Date', parse_dates=True, infer_datetime_format=True)
weight

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY
Date,,,,,,,,,,,
2001-03-30,1.2704,5.5231,NaN,43.1115,2.5412,NaN,43.5279,NaN,0.5485,NaN,3.4773
2001-06-29,1.3049,5.7885,NaN,42.1752,2.4165,1.7111,41.5455,NaN,NaN,NaN,4.8921
2001-09-28,1.1416,5.6963,NaN,38.1666,2.1217,1.6766,46.0285,NaN,0.0832,NaN,4.9522
2001-12-31,1.1057,5.5975,NaN,39.3557,2.2442,1.3093,44.3632,NaN,0.3818,NaN,5.6426
2002-03-29,0.7209,5.4681,NaN,40.1715,2.4241,1.3339,44.2882,NaN,NaN,NaN,5.1763
2002-06-28,1.0026,4.7015,NaN,39.0144,2.3803,0.9466,47.1022,NaN,NaN,NaN,4.8523
2002-09-30,0.8122,5.7693,0.0070,40.4428,2.4897,0.0496,45.6390,NaN,NaN,NaN,4.7905
2002-12-31,1.0221,6.2341,0.0154,40.9932,3.5228,1.4688,41.8464,NaN,NaN,NaN,4.8972
2003-03-31,1.0486,6.1783,0.0548,40.8024,3.7573,1.8218,41.0535,NaN,0.2560,NaN,5.0273


## Reset the rebalancing dates

We are to replicate a real time portfolio weightings, which means we get to know about the latest rebalancing weightings only after 45 days from the end of each quarter when the due dates of 13F filings fall on. For example, we could know about the exact sector weightings of a fund at the 2nd quarter end of 2018(2018-06-30) on or after 2018-08-15. Regarding this, we assume that we reset rebalancing process at every 45 days + 1 business day from each quarter end. Be sure to check if the rebalancing day falls on a business day.    

In [89]:
weight = weight.loc['2018-06-29':]
weight.head()

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY
Date,,,,,,,,,,,
2018-06-29,0.3760,3.9663,1.9939,40.4963,4.9049,23.8424,19.9746,0.2608,0.0265,2.0027,2.1549
2018-09-28,0.3201,3.6832,0.7871,42.5871,5.1656,26.7529,16.9625,0.2341,0.0256,1.6918,1.7898
2018-12-31,0.3104,3.8247,0.7242,45.5183,4.9520,21.9066,18.5125,0.2880,0.0215,1.4715,2.4705
2019-03-29,0.3066,3.7933,0.4398,46.4856,4.8086,24.2341,15.2830,0.3127,0.0263,1.4125,2.8975
2019-06-28,0.3471,3.9006,0.4107,47.3721,4.8536,24.2044,15.2266,0.2970,0.0225,1.2563,2.1090


In [90]:
df1= weight.resample('Q').mean()
df1

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY
Date,,,,,,,,,,,
2018-06-30,0.3760,3.9663,1.9939,40.4963,4.9049,23.8424,19.9746,0.2608,0.0265,2.0027,2.1549
2018-09-30,0.3201,3.6832,0.7871,42.5871,5.1656,26.7529,16.9625,0.2341,0.0256,1.6918,1.7898
2018-12-31,0.3104,3.8247,0.7242,45.5183,4.9520,21.9066,18.5125,0.2880,0.0215,1.4715,2.4705
2019-03-31,0.3066,3.7933,0.4398,46.4856,4.8086,24.2341,15.2830,0.3127,0.0263,1.4125,2.8975
2019-06-30,0.3471,3.9006,0.4107,47.3721,4.8536,24.2044,15.2266,0.2970,0.0225,1.2563,2.1090
2019-09-30,0.3408,3.7923,0.5602,46.0393,4.7082,25.9615,14.9960,0.3245,0.0216,1.1835,2.0720
2019-12-31,0.3047,3.6987,0.5363,43.5571,4.2145,29.7417,14.2518,0.2865,0.0216,1.4697,1.9173
2020-03-31,0.2368,4.1155,0.2595,37.1991,2.9493,35.5163,15.7388,0.1922,0.0158,2.0100,1.7667
2020-06-30,0.5466,4.0496,0.1599,32.0195,0.0033,44.1837,15.0092,0.2872,0.0126,1.8579,1.8705


In [91]:
df1.index = df1.index + timedelta(days=45) 
df1

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY
Date,,,,,,,,,,,
2018-08-14,0.3760,3.9663,1.9939,40.4963,4.9049,23.8424,19.9746,0.2608,0.0265,2.0027,2.1549
2018-11-14,0.3201,3.6832,0.7871,42.5871,5.1656,26.7529,16.9625,0.2341,0.0256,1.6918,1.7898
2019-02-14,0.3104,3.8247,0.7242,45.5183,4.9520,21.9066,18.5125,0.2880,0.0215,1.4715,2.4705
2019-05-15,0.3066,3.7933,0.4398,46.4856,4.8086,24.2341,15.2830,0.3127,0.0263,1.4125,2.8975
2019-08-14,0.3471,3.9006,0.4107,47.3721,4.8536,24.2044,15.2266,0.2970,0.0225,1.2563,2.1090
2019-11-14,0.3408,3.7923,0.5602,46.0393,4.7082,25.9615,14.9960,0.3245,0.0216,1.1835,2.0720
2020-02-14,0.3047,3.6987,0.5363,43.5571,4.2145,29.7417,14.2518,0.2865,0.0216,1.4697,1.9173
2020-05-15,0.2368,4.1155,0.2595,37.1991,2.9493,35.5163,15.7388,0.1922,0.0158,2.0100,1.7667
2020-08-14,0.5466,4.0496,0.1599,32.0195,0.0033,44.1837,15.0092,0.2872,0.0126,1.8579,1.8705


In [92]:
from pandas.tseries.offsets import *
df1.index = df1.index + BDay()
df1

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY
Date,,,,,,,,,,,
2018-08-15,0.3760,3.9663,1.9939,40.4963,4.9049,23.8424,19.9746,0.2608,0.0265,2.0027,2.1549
2018-11-15,0.3201,3.6832,0.7871,42.5871,5.1656,26.7529,16.9625,0.2341,0.0256,1.6918,1.7898
2019-02-15,0.3104,3.8247,0.7242,45.5183,4.9520,21.9066,18.5125,0.2880,0.0215,1.4715,2.4705
2019-05-16,0.3066,3.7933,0.4398,46.4856,4.8086,24.2341,15.2830,0.3127,0.0263,1.4125,2.8975
2019-08-15,0.3471,3.9006,0.4107,47.3721,4.8536,24.2044,15.2266,0.2970,0.0225,1.2563,2.1090
2019-11-15,0.3408,3.7923,0.5602,46.0393,4.7082,25.9615,14.9960,0.3245,0.0216,1.1835,2.0720
2020-02-17,0.3047,3.6987,0.5363,43.5571,4.2145,29.7417,14.2518,0.2865,0.0216,1.4697,1.9173
2020-05-18,0.2368,4.1155,0.2595,37.1991,2.9493,35.5163,15.7388,0.1922,0.0158,2.0100,1.7667
2020-08-17,0.5466,4.0496,0.1599,32.0195,0.0033,44.1837,15.0092,0.2872,0.0126,1.8579,1.8705


In [93]:
df1.index[6]

Timestamp('2020-02-17 00:00:00')

In [94]:
b_day = pd.Timestamp('2020-02-18')
b_day

Timestamp('2020-02-18 00:00:00')

In [95]:
weight_df = df1.rename(index = {df1.index[6]:b_day})

In [96]:
weight_df

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY
Date,,,,,,,,,,,
2018-08-15,0.3760,3.9663,1.9939,40.4963,4.9049,23.8424,19.9746,0.2608,0.0265,2.0027,2.1549
2018-11-15,0.3201,3.6832,0.7871,42.5871,5.1656,26.7529,16.9625,0.2341,0.0256,1.6918,1.7898
2019-02-15,0.3104,3.8247,0.7242,45.5183,4.9520,21.9066,18.5125,0.2880,0.0215,1.4715,2.4705
2019-05-16,0.3066,3.7933,0.4398,46.4856,4.8086,24.2341,15.2830,0.3127,0.0263,1.4125,2.8975
2019-08-15,0.3471,3.9006,0.4107,47.3721,4.8536,24.2044,15.2266,0.2970,0.0225,1.2563,2.1090
2019-11-15,0.3408,3.7923,0.5602,46.0393,4.7082,25.9615,14.9960,0.3245,0.0216,1.1835,2.0720
2020-02-18,0.3047,3.6987,0.5363,43.5571,4.2145,29.7417,14.2518,0.2865,0.0216,1.4697,1.9173
2020-05-18,0.2368,4.1155,0.2595,37.1991,2.9493,35.5163,15.7388,0.1922,0.0158,2.0100,1.7667
2020-08-17,0.5466,4.0496,0.1599,32.0195,0.0033,44.1837,15.0092,0.2872,0.0126,1.8579,1.8705


In [97]:
dollar_value = pd.DataFrame(index= df.index, columns=df.columns)

In [98]:
initial_capital = 100000
# Setting null value if not purchased at the start date.
for j in range(len(weight_df.index)):
    for i in range(len(df.index)):
       
        for index, col in df.iterrows():
            if  j == 0 and df.index[i] == weight_df.index[j]:
                etf_portfolio.loc['purchase_date'] = weight_df.index[j].date()
                etf_portfolio.loc['weights'] = weight_df.loc[weight_df.index[j]]
                etf_portfolio.loc['seed_money'] = etf_portfolio.loc['weights'] * initial_capital / 100
                etf_portfolio.loc["num_shares"] = etf_portfolio.loc["seed_money"] / df.loc[df.index[i]]  
                
                # dollar value calculation
                value_by_stock = df.loc[df.index[i]] * etf_portfolio.loc["num_shares"]
                dollar_value.loc[dollar_value.index[i]] = value_by_stock
            
            elif j == 0 and df.index[i] != weight_df.index[j]:
                value_by_stock = df.loc[df.index[i]] * etf_portfolio.loc["num_shares"]
                dollar_value.loc[dollar_value.index[i]] = value_by_stock
                
            elif j != 0 and df.index[i] == weight_df.index[j]:
                etf_portfolio.index.insert(-1, f"rebalacning_date_{j}")
                etf_portfolio.index.insert(-1, f"weights_{j}")
                etf_portfolio.index.insert(-1, f"seed_money_{j}")
                etf_portfolio.index.insert(-1, f"num_shares_{j}")
                
                etf_portfolio.loc[f"rebalacning_date_{j}"] = weight_df.index[j].date()
                etf_portfolio.loc[f'weights_{j}'] = weight_df.loc[weight_df.index[j]]
                total_portfolio_value = dollar_value.iloc[i].sum()
                etf_portfolio.loc[f"seed_money_{j}"] = etf_portfolio.loc[f'weights_{j}'] * total_portfolio_value / 100
                etf_portfolio.loc[f"num_shares_{j}"] = etf_portfolio.loc[f"seed_money_{j}"] / df.loc[df.index[i]]  
                
                # dollar value calculation
                value_by_stock = df.loc[df.index[i]] * etf_portfolio.loc[f"num_shares_{j}"]
                dollar_value.loc[dollar_value.index[i]] = value_by_stock
                
            elif j!= 0 and df.index[i] > weight_df.index[j]:
                value_by_stock = df.loc[df.index[i]] * etf_portfolio.loc[f"num_shares_{j}"]
                dollar_value.loc[dollar_value.index[i]] = value_by_stock  

In [99]:
etf_portfolio

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY
purchase_date,2018-08-15,2018-08-15,2018-08-15,2018-08-15,2018-08-15,2018-08-15,2018-08-15,2018-08-15,2018-08-15,2018-08-15,2018-08-15
weights,0.376,3.9663,1.9939,40.4963,4.9049,23.8424,19.9746,0.2608,0.0265,2.0027,2.1549
seed_money,376,3966.3,1993.9,40496.3,4904.9,23842.4,19974.6,260.8,26.5,2002.7,2154.9
num_shares,6.52325,81.1768,27.7277,1459.33,65.5035,327.731,371.206,7.82244,0.494311,22.2671,19.2643
rebalacning_date_1,2018-11-15,2018-11-15,2018-11-15,2018-11-15,2018-11-15,2018-11-15,2018-11-15,2018-11-15,2018-11-15,2018-11-15,2018-11-15
weights_1,0.3201,3.6832,0.7871,42.5871,5.1656,26.7529,16.9625,0.2341,0.0256,1.6918,1.7898
seed_money_1,310.609,3574,763.763,41324.4,5012.45,25959.7,16459.6,227.159,24.841,1641.64,1736.73
num_shares_1,5.67842,79.2812,11.4903,1544.26,69.6173,379.972,294.466,6.88361,0.459253,18.0044,16.26
rebalacning_date_2,2019-02-15,2019-02-15,2019-02-15,2019-02-15,2019-02-15,2019-02-15,2019-02-15,2019-02-15,2019-02-15,2019-02-15,2019-02-15
weights_2,0.3104,3.8247,0.7242,45.5183,4.952,21.9066,18.5125,0.288,0.0215,1.4715,2.4705


In [106]:
dollar_value = dollar_value.loc['2018-08-15':]

In [107]:
portfolio_value = dollar_value.copy()
portfolio_value["Daily Total"] = round(dollar_value[etf_portfolio.columns].sum(axis=1), 2)
portfolio_value

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY,Daily Total
Date,,,,,,,,,,,,
2018-08-15,376,3966.3,1993.9,40496.3,4904.9,23842.4,19974.6,260.8,26.5,2002.7,2154.9,99999.30
2018-08-16,378.414,3964.68,2008.32,41007.1,4965.16,23934.2,20279,262.912,26.8164,2021.85,2165.3,101013.67
2018-08-17,381.056,3946.01,2013.86,41094.6,4995.95,23983.3,20427.5,265.337,26.9449,2028.98,2167.42,101330.97
2018-08-20,383.763,3949.25,2027.17,41226,5028.05,23944,20416.3,265.533,26.851,2035.66,2181.1,101483.67
2018-08-21,385.263,3976.85,2037.43,41386.5,5066.04,23970.2,20256.7,263.186,26.6532,2034.99,2199.98,101603.81
2018-08-22,383.502,3973.61,2061.83,41284.3,5018.88,24042.3,20137.9,261.582,26.4555,2037.21,2202.87,101430.52
2018-08-23,380.827,3955.75,2051.57,41080,5003.16,24091.5,20089.7,261.035,26.4506,2035.66,2199.4,101175.02
2018-08-24,385.459,4010.95,2067.38,41196.8,5024.12,24340.6,20063.7,262.677,26.5643,2044.78,2212.11,101635.06
2018-08-27,390.938,4053.16,2082.49,41736.7,5085.03,24566.7,20074.8,262.443,26.3913,2052.58,2227.53,102558.80


In [108]:
portfolio_daily_return = portfolio_value.pct_change()
portfolio_daily_return

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY,Daily Total
Date,,,,,,,,,,,,
2018-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-08-16,0.006419,-0.000409,0.007231,0.012613,0.012286,0.003849,0.015239,0.008098,0.011938,0.009562,0.004827,0.010144
2018-08-17,0.006982,-0.004709,0.002761,0.002135,0.006201,0.002054,0.007322,0.009223,0.004793,0.003524,0.000979,0.003141
2018-08-20,0.007104,0.000823,0.006609,0.003196,0.006425,-0.001640,-0.000545,0.000737,-0.003486,0.003292,0.006311,0.001507
2018-08-21,0.003910,0.006989,0.005061,0.003894,0.007556,0.001095,-0.007818,-0.008838,-0.007364,-0.000328,0.008656,0.001184
2018-08-22,-0.004572,-0.000816,0.011976,-0.002468,-0.009310,0.003008,-0.005864,-0.006093,-0.007418,0.001094,0.001313,-0.001706
2018-08-23,-0.006974,-0.004494,-0.004976,-0.004949,-0.003132,0.002045,-0.002396,-0.002093,-0.000187,-0.000765,-0.001574,-0.002519
2018-08-24,0.012162,0.013954,0.007704,0.002842,0.004190,0.010339,-0.001293,0.006293,0.004298,0.004485,0.005781,0.004547
2018-08-27,0.014216,0.010524,0.007310,0.013107,0.012125,0.009290,0.000555,-0.000893,-0.006513,0.003811,0.006967,0.009089


In [109]:
cumulative_portfolio_return = (1 + portfolio_daily_return).cumprod().dropna()
cumulative_portfolio_return

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY,Daily Total
Date,,,,,,,,,,,,
2018-08-16,1.006419,0.999591,1.007231,1.012613,1.012286,1.003849,1.015239,1.008098,1.011938,1.009562,1.004827,1.010144
2018-08-17,1.013446,0.994883,1.010013,1.014775,1.018563,1.005911,1.022672,1.017397,1.016788,1.013120,1.005811,1.013317
2018-08-20,1.020645,0.995702,1.016688,1.018018,1.025107,1.004261,1.022115,1.018146,1.013244,1.016455,1.012158,1.014844
2018-08-21,1.024636,1.002661,1.021833,1.021982,1.032853,1.005361,1.014124,1.009148,1.005783,1.016122,1.020919,1.016045
2018-08-22,1.019951,1.001842,1.034070,1.019459,1.023237,1.008385,1.008177,1.002999,0.998321,1.017234,1.022260,1.014312
2018-08-23,1.012838,0.997339,1.028925,1.014414,1.020032,1.010447,1.005761,1.000900,0.998135,1.016455,1.020651,1.011757
2018-08-24,1.025156,1.011257,1.036852,1.017297,1.024306,1.020893,1.004460,1.007199,1.002425,1.021014,1.026551,1.016358
2018-08-27,1.039729,1.021899,1.044431,1.030631,1.036725,1.030378,1.005018,1.006299,0.995896,1.024905,1.033703,1.025595
2018-08-28,1.036086,1.017059,1.039911,1.030090,1.035524,1.032165,1.002044,1.018296,0.994031,1.026017,1.035759,1.024951


In [110]:
dollar_value.to_csv("berkshire_dollar_value_since_2Q2018_real_port.csv")
etf_portfolio.to_csv("berkshire_etf_portfolio_2Q2018_real_port.csv")
portfolio_value.to_csv("berkshire_portfolio_value_2Q2018_real_port.csv")